In [14]:
# Data Cleaning This notebook cleans and prepares Formula 1 race and qualifying data for analysis. The goal is to create analysis-ready tables while preserving the integrity of the original raw datasets.

In [15]:
import pandas as pd

DATA_PATH = "../data/raw/"

drivers = pd.read_csv(DATA_PATH + "drivers.csv")
races = pd.read_csv(DATA_PATH + "races.csv")
results = pd.read_csv(DATA_PATH + "results.csv")
qualifying = pd.read_csv(DATA_PATH + "qualifying.csv")
constructors = pd.read_csv(DATA_PATH + "constructors.csv")

In [16]:
results_clean = results.copy()

# Convert position to numeric (invalid → NaN)
results_clean['position'] = pd.to_numeric(
    results_clean['position'], errors='coerce'
)

# Keep only classified finishers
results_clean = results_clean.dropna(subset=['position'])

# Convert to integer
results_clean['position'] = results_clean['position'].astype(int)

print(f"Clean results rows: {len(results_clean)}")


Clean results rows: 16285


In [17]:
qualifying_clean = qualifying.copy()

qualifying_clean['position'] = pd.to_numeric(
    qualifying_clean['position'], errors='coerce'
)

qualifying_clean = qualifying_clean.dropna(subset=['position'])
qualifying_clean['position'] = qualifying_clean['position'].astype(int)

print(f"Clean qualifying rows: {len(qualifying_clean)}")

Clean qualifying rows: 10973


In [18]:
drivers_clean = drivers[['driverId','forename','surname','nationality']]

constructors_clean = constructors[['constructorId','name']]

races_clean = races[['raceId','year','name','circuitId']]

In [19]:
race_data = (
    results_clean
    .merge(races_clean, on='raceId', how='left')
    .merge(drivers_clean, on='driverId', how='left')
    .merge(constructors_clean, on='constructorId', how='left')
    .merge(
        qualifying_clean[['raceId','driverId','position']],
        on=['raceId', 'driverId'],
        how='left',
        suffixes=('','_qualifying')
    )
)

race_data.rename(columns={'position_qualifying': 'grid_position'}, inplace=True)

In [20]:
race_data.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,fastestLapSpeed,statusId,year,name_x,circuitId,forename,surname,nationality,name_y,grid_position
0,1,18,1,1,22,1,1,1,1,10.0,...,218.3,1,2008,Australian Grand Prix,1,Lewis,Hamilton,British,McLaren,1.0
1,2,18,2,2,3,5,2,2,2,8.0,...,217.586,1,2008,Australian Grand Prix,1,Nick,Heidfeld,German,BMW Sauber,5.0
2,3,18,3,3,7,7,3,3,3,6.0,...,216.719,1,2008,Australian Grand Prix,1,Nico,Rosberg,German,Williams,7.0
3,4,18,4,4,5,11,4,4,4,5.0,...,215.464,1,2008,Australian Grand Prix,1,Fernando,Alonso,Spanish,Renault,12.0
4,5,18,5,1,23,3,5,5,5,4.0,...,218.385,1,2008,Australian Grand Prix,1,Heikki,Kovalainen,Finnish,McLaren,3.0


In [21]:
OUTPUT_PATH = "../data/processed/"

race_data.to_csv(OUTPUT_PATH + "race_data_clean.csv", index=False)